### Memuat Modul dan Membuat Prediksi Menggunakan Model TensorFlow Serving

Kode ini mengimpor modul yang diperlukan untuk memproses data, mengonversi data ke dalam format yang dapat dipahami oleh TensorFlow Serving, dan mengirimkan permintaan prediksi ke model yang di-hosting. Modul yang dimuat mencakup `base64` dan `json` untuk encoding dan decoding data, `pandas` untuk manipulasi data, `requests` untuk mengirim permintaan HTTP, dan `tensorflow` untuk fungsi terkait TensorFlow.

In [43]:
import base64
import json
import random
import pandas as pd
import requests
import tensorflow as tf

### Membaca Data CSV dan Menyiapkan Input untuk Prediksi

Kode ini membaca data dari file CSV `AIDS_Classification.csv`, memisahkan kolom label `infected`, dan memilih secara acak satu baris data untuk digunakan sebagai fitur input. Baris data yang dipilih diubah menjadi dictionary yang berisi pasangan kunci-nilai dari nama kolom dan nilai fitur, siap untuk digunakan dalam prediksi model.

In [44]:
df = pd.read_csv("data/AIDS_Classification.csv")
y = df.pop("infected")

columns = df.columns.values
rand = random.randint(0, len(df))

features = df.values[rand]
label = y[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'time': 866.0,
 'trt': 2.0,
 'age': 33.0,
 'wtkg': 70.7616,
 'hemo': 0.0,
 'homo': 1.0,
 'drugs': 0.0,
 'karnof': 100.0,
 'oprior': 0.0,
 'z30': 0.0,
 'preanti': 0.0,
 'race': 1.0,
 'gender': 1.0,
 'str2': 0.0,
 'strat': 1.0,
 'symptom': 0.0,
 'treat': 1.0,
 'offtrt': 0.0,
 'cd40': 513.0,
 'cd420': 576.0,
 'cd80': 1512.0,
 'cd820': 1368.0}

### Fungsi Konversi Tipe Data ke Format TensorFlow Feature

Kode ini mendefinisikan tiga fungsi (`string_feature`, `float_feature`, dan `int_feature`) yang masing-masing mengonversi nilai string, float, dan integer menjadi objek `tf.train.Feature`. Objek `tf.train.Feature` ini kemudian dapat digunakan untuk membuat protokol buffer TensorFlow (`tf.train.Example`) yang diperlukan untuk melakukan prediksi dengan model TensorFlow Serving.

In [45]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[bytes(value, "utf-8")]),
    )


def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(value=[value]),
    )


def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(value=[value]),
    )

### Menyiapkan JSON untuk Prediksi dengan TensorFlow Serving

Kode ini mendefinisikan fungsi `prepare_json` yang mengonversi dictionary input menjadi format JSON yang dapat diterima oleh TensorFlow Serving. Fitur-fitur input dikonversi menjadi objek `tf.train.Feature`, dikemas menjadi protokol buffer `tf.train.Example`, dan dienkode dalam base64 sebelum dikemas dalam format JSON untuk dikirim sebagai permintaan prediksi.

In [46]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if keys == "wtkg":
            feature_spec[keys] = float_feature(float(values))
        else:
            feature_spec[keys] = int_feature(int(values))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{"examples": {"b64": base64.b64encode(example).decode()}}]

    return json.dumps(
        {
            "signature_name": "serving_default",
            "instances": result,
        }
    )

### Mengirimkan Input untuk Prediksi dan Mengembalikan Hasil

Kode ini mendefinisikan fungsi `make_predictions` yang menyiapkan data input dalam format JSON menggunakan `prepare_json` dan mengirimkan permintaan POST ke endpoint model TensorFlow Serving. Fungsi ini mengembalikan hasil prediksi dengan mengevaluasi respons dari model, mengklasifikasikan hasil sebagai "Not Infected" atau "Infected" berdasarkan nilai prediksi.

In [47]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)

    endpoint = (
        "https://jericholuthfiihfk-pipeline-production.up.railway.app/v1/models/aids-model:predict"
    )
    response = requests.post(endpoint, data=json_data)
    prediction = response.json()["predictions"][0][0]

    if prediction < 0.6:
        return "Not Infected"
    else:
        return "Infected"
    
make_predictions(inputs)

'Infected'

### Menentukan Label dari Prediksi Model

Variabel `label` menyimpan nilai yang menunjukkan kondisi aktual (ground truth) dari contoh data yang dipilih secara acak dari dataset, yaitu apakah individu tersebut terinfeksi atau tidak. Nilai ini digunakan untuk memeriksa akurasi prediksi yang dihasilkan oleh model.

In [48]:
label

1